# Jupiter Migration Simulation

**Purpose**: Study how Jupiter's migration affects L4/L5 Trojan populations.

This notebook:
1. Loads a burn-in state (relaxed particle distribution)
2. Applies migration physics to Jupiter (inward or outward)
3. Tracks L4/L5 population evolution
4. Computes ratio slopes for comparison

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '.')

from simulation import (
    SimulationConfig,
    run_simulation,
    load_checkpoint,
    plot_trojan_evolution,
    plot_snapshot,
    create_animation,
    compute_population_summary,
    compute_jupiter_orbit
)

print('Simulation module loaded successfully!')

## 2. Configuration

Choose migration direction and parameters:

In [ ]:
# === MIGRATION CONFIGURATION ===

# Migration direction: 'inward' or 'outward'
DIRECTION = 'inward'

# Migration duration in thousands of years
DURATION_KYR = 50

# Semi-major axis range (AU)
if DIRECTION == 'inward':
    INITIAL_A = 5.6  # Start further out
    TARGET_A = 5.2   # Migrate inward to current position
else:  # outward
    INITIAL_A = 4.8  # Start further in
    TARGET_A = 5.2   # Migrate outward to current position

# Input file (burn-in state to continue from)
# Set to the appropriate burn-in file for your initial_a
INPUT_FILE = f'jupiter_burnin_5kyr_{INITIAL_A}au.pkl'

# Number of snapshots
N_SNAPSHOTS = 200

# Create configuration
if DIRECTION == 'inward':
    config = SimulationConfig.migration_inward(
        duration_kyr=DURATION_KYR,
        initial_a=INITIAL_A,
        target_a=TARGET_A,
        input_file=INPUT_FILE,
        n_snapshots=N_SNAPSHOTS
    )
else:
    config = SimulationConfig.migration_outward(
        duration_kyr=DURATION_KYR,
        initial_a=INITIAL_A,
        target_a=TARGET_A,
        input_file=INPUT_FILE,
        n_snapshots=N_SNAPSHOTS
    )

print(f'Configuration: {config.name}')
print(f'  Direction: {DIRECTION}')
print(f'  Duration: {config.end_time:.0f} years')
print(f'  Jupiter: {INITIAL_A} AU → {TARGET_A} AU')
print(f'  Migration tau_a: {config.migration.tau_a:.0f} years')
print(f'  Input: {INPUT_FILE}')
print(f'  Output: {config.output_file}')

## 3. Run Migration Simulation

In [ ]:
# Run the simulation
results = run_simulation(config, verbose=True)

## 4. Analysis

In [ ]:
# Extract results
times = results['times']
l4_counts = results['l4_counts']
l5_counts = results['l5_counts']
snapshots_massive = results['snapshots_massive']
snapshots_planetesimals = results['snapshots_planetesimals']

print(f'Loaded {len(times)} snapshots')
print(f'Time range: {times[0]:.1f} - {times[-1]:.1f} yr')

In [ ]:
# Track Jupiter's orbital evolution
import numpy as np
import matplotlib.pyplot as plt

jupiter_a = []
jupiter_e = []

for snap in snapshots_massive:
    orbit = compute_jupiter_orbit(snap)
    jupiter_a.append(orbit['a'])
    jupiter_e.append(orbit['e'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Semi-major axis evolution
ax1.plot(times, jupiter_a, 'b-', linewidth=2)
ax1.axhline(INITIAL_A, color='green', linestyle='--', label=f'Initial ({INITIAL_A} AU)')
ax1.axhline(TARGET_A, color='red', linestyle='--', label=f'Target ({TARGET_A} AU)')
ax1.set_xlabel('Time (yr)')
ax1.set_ylabel('Semi-major axis (AU)')
ax1.set_title('Jupiter Migration')
ax1.legend()
ax1.grid(alpha=0.3)

# Eccentricity evolution
ax2.plot(times, jupiter_e, 'r-', linewidth=2)
ax2.set_xlabel('Time (yr)')
ax2.set_ylabel('Eccentricity')
ax2.set_title('Jupiter Eccentricity')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{config.name}_jupiter.png', dpi=150)
plt.show()

print(f'Jupiter: {jupiter_a[0]:.3f} AU → {jupiter_a[-1]:.3f} AU')
print(f'Eccentricity: {jupiter_e[0]:.4f} → {jupiter_e[-1]:.4f}')

In [ ]:
# Plot Trojan population evolution
fig = plot_trojan_evolution(
    times, l4_counts, l5_counts,
    title=f'Trojan Evolution ({config.name})'
)
fig.savefig(f'{config.name}_evolution.png', dpi=150)
print(f'Saved: {config.name}_evolution.png')

In [ ]:
# Population summary
summary = compute_population_summary(times, l4_counts, l5_counts)

print('=' * 60)
print(f'MIGRATION SUMMARY: {DIRECTION.upper()}')
print('=' * 60)
print(f"Duration: {DURATION_KYR} kyr")
print(f"Jupiter: {INITIAL_A} → {TARGET_A} AU")
print()
print(f"Initial: L4={summary['initial_l4']}, L5={summary['initial_l5']}, Total={summary['initial_total']}")
print(f"Final:   L4={summary['final_l4']}, L5={summary['final_l5']}, Total={summary['final_total']}")
print()
print(f"Initial L4/L5 ratio: {summary['initial_ratio']:.3f}")
print(f"Final L4/L5 ratio:   {summary['final_ratio']:.3f}")
print(f"L4/L5 ratio slope:   {summary['ratio_slope']:.3e} yr⁻¹")
print()
print(f"Retention: {summary['retention_percent']:.1f}%")
print('=' * 60)

# Save slope for comparison
slope_label = f"{DURATION_KYR}kyr_{DIRECTION[:2]}"
print(f"\nFor slopes.md: {slope_label}: {summary['ratio_slope']:.3e} yr⁻¹")

## 5. Batch Mode: Run Multiple Configurations

Use this section to run multiple migration scenarios and compare results.

In [ ]:
# Define multiple scenarios to run
SCENARIOS = [
    # (direction, duration_kyr, initial_a, target_a, input_file)
    ('inward', 50, 5.6, 5.2, 'jupiter_burnin_5kyr_5.6au.pkl'),
    ('inward', 100, 5.6, 5.2, 'jupiter_burnin_5kyr_5.6au.pkl'),
    ('outward', 50, 4.8, 5.2, 'jupiter_burnin_5kyr_4.8au.pkl'),
    ('outward', 100, 4.8, 5.2, 'jupiter_burnin_5kyr_4.8au.pkl'),
]

print(f'Defined {len(SCENARIOS)} scenarios for batch processing')
for i, (d, dur, a0, af, inp) in enumerate(SCENARIOS, 1):
    print(f'  {i}. {d} {dur}kyr: {a0} → {af} AU (from {inp})')

In [ ]:
# Run all scenarios (uncomment to execute)
# WARNING: This will take a long time!

# all_results = []
# for direction, duration, initial_a, target_a, input_file in SCENARIOS:
#     if direction == 'inward':
#         cfg = SimulationConfig.migration_inward(
#             duration_kyr=duration,
#             initial_a=initial_a,
#             target_a=target_a,
#             input_file=input_file
#         )
#     else:
#         cfg = SimulationConfig.migration_outward(
#             duration_kyr=duration,
#             initial_a=initial_a,
#             target_a=target_a,
#             input_file=input_file
#         )
#     
#     print(f'\n{"="*60}')
#     print(f'Running: {cfg.name}')
#     print(f'{"="*60}')
#     
#     result = run_simulation(cfg, verbose=True)
#     all_results.append((cfg.name, result))

print('Batch mode ready. Uncomment the code above to run all scenarios.')

## 6. Final Visualization

In [ ]:
# Plot final snapshot
fig = plot_snapshot(
    snapshots_massive[-1],
    snapshots_planetesimals[-1],
    times[-1],
    show_trojan_zones=True
)
fig.savefig(f'{config.name}_final.png', dpi=150)
print(f'Saved: {config.name}_final.png')

In [ ]:
# Create animation (optional)
gif_file = create_animation(
    times,
    snapshots_massive,
    snapshots_planetesimals,
    output_file=f'{config.name}.gif',
    fps=15,
    dpi=80
)

from IPython.display import Image
Image(filename=gif_file)